In [1]:
#default_exp interference

In [2]:
import sys
#sys.path.append("/home/qhs67/git/bachelorthesis_sven_thaele/code/")
sys.path.insert(0, "../pointpillars")

In [3]:
#export
import torch
import logging

from data.dataset import VelTrainDataset
from modules.pointpillars import PointPillars, init_weights
from utils.io import read_config, load_network_save, load_single_pcd
from utils.viewer.pcviewer import PCViewer
from utils.math.box_ops import bb_tensor_to_velodyne_coords

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
#export
logger = logging.getLogger(__name__)


In [5]:
#export
def predict_single_pcd(pcd_nbr: int, net_save_name: str):
    """Create a network prediction from a saved network for a single point cloud"""

    vel_folder = "/home/qhs67/git/bachelorthesis_sven_thaele/code/data/kitti/training/velodyne/training"
    label_folder ="/home/qhs67/git/bachelorthesis_sven_thaele/code/data/kitti/training/label_2/training"

    dataset = VelTrainDataset(vel_folder, label_folder)
    batch_tuple = dataset[pcd_nbr]
    
    batch = []
    for i, tensor in enumerate(batch_tuple):
        tensor = tensor.unsqueeze(0)
        batch.append(tensor)
        
    pil_batch, ind_batch, label_batch = batch

    with torch.no_grad():
        conf = read_config()
        net_save = load_network_save(net_save_name)
        net = PointPillars(conf)
        net.load_state_dict(net_save)
        net.eval()
        net.cuda()

        pred_anchors= net(pil_batch, ind_batch, label_batch)

    return pred_anchors, label_batch

In [6]:
#export
def view_single_pcd_prediction(pcd_nbr: str, net_save_name: str):
    pcloud = load_single_pcd(pcd_nbr)
    pred_anchors, label_batch= predict_single_pcd(int(pcd_nbr), net_save_name)

    label_batch = label_batch[0]
    label_batch = bb_tensor_to_velodyne_coords(label_batch)

    PCViewer.view_pcd_from_network_output(pcloud, pred_anchors, label_batch)



In [7]:
pcloud = load_single_pcd('002137')
pcloud

tensor([[ 7.7512e+01,  4.7000e-02,  2.8300e+00,  0.0000e+00],
        [ 5.2519e+01,  8.8510e+00,  2.0100e+00,  1.6000e-01],
        [ 5.2292e+01,  8.9810e+00,  2.0030e+00,  1.0000e-01],
        ...,
        [ 3.8420e+00, -1.4470e+00, -1.7980e+00,  3.9000e-01],
        [ 3.8530e+00, -1.4380e+00, -1.8020e+00,  0.0000e+00],
        [ 3.8750e+00, -1.4320e+00, -1.8100e+00,  0.0000e+00]], device='cuda:0')

In [8]:
torch.min(pcloud, dim=0), torch.max(pcloud, dim=0)

(torch.return_types.min(
 values=tensor([-71.8370, -55.1200,  -8.8140,   0.0000], device='cuda:0'),
 indices=tensor([ 13281,   1474, 113923,      0], device='cuda:0')),
 torch.return_types.max(
 values=tensor([79.8020, 32.9710,  2.8300,  0.9900], device='cuda:0'),
 indices=tensor([ 6116, 10917,     0, 28486], device='cuda:0')))

In [9]:
#view_single_pcd_prediction("003009", "trained_22-06-2021-19-43-55_e10000")
view_single_pcd_prediction("000011", "trained_28-06-2021-13-18-04_e80")

INFO - 2021-07-02 10:44:36,954 - head - Initialize detection head...
INFO - 2021-07-02 10:44:36,957 - boxmatch - Initializing BoxMatch module...


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!